In [116]:
import pandas as pd
from scipy.spatial import distance
from sqlalchemy import create_engine

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [117]:
import os

import pymysql
import pandas as pd

from sqlalchemy import create_engine
from dotenv import load_dotenv


In [118]:
# setup env
load_dotenv()

True

In [119]:
df_feature_vector = pd.read_parquet('data/feature_vector_num.parquet')
df_feature_vector_with_text = pd.read_parquet('data/feature_vector.parquet')
df_tag = pd.read_parquet('data/tag.parquet')
df_feature_vector_with_text.shape

(134, 80)

In [120]:
# setup db
user = os.getenv('RDS_USERNAME')
password = os.getenv('RDS_PASSWORD')
host = os.getenv('RDS_HOSTNAME')
db = os.getenv('RDS_DB_NAME')
port = os.getenv('RDS_PORT')

db_string = "postgresql://{}:{}@{}:{}/{}".format(user,password,host,port,db)
sqlEngine = create_engine(db_string)

In [121]:
df_tag = pd.read_parquet('data/tag.parquet')
df_tag = df_tag[['name', 'category_id']]
df_tag['weight'] = df_tag['category_id'].apply(lambda x: 3 if x == 9 else 0.3 if x == 1 else 1)

In [122]:
df_all_features = pd.DataFrame(df_feature_vector.columns.to_list(), columns=['name'])

In [123]:
df_merge_tag_features = pd.merge(df_tag, df_all_features, on='name')

allFeatures = df_feature_vector.columns.to_list()
df_name = df_merge_tag_features['name']

for i in df_name.to_numpy():
    allFeatures.remove(i)

for i in allFeatures:
    df_merge_tag_features = df_merge_tag_features.append({'name': i, 'weight': 0.3}, ignore_index=True)

array = df_feature_vector.columns.to_list()
array.remove('id')
array.remove('Mit Abstand')
array.remove('Videokonferenz')
array.remove('Pfadfinder')
array.remove('Rover')

weight = df_merge_tag_features['weight'].to_numpy()
array_labels = df_merge_tag_features['name'].to_numpy()

134

In [124]:
heimabend_index = 5

for heimabend_index in range(0, df_feature_vector.shape[0]):
    print('heimabend_index')
    print(heimabend_index)
    print('heimabend_index')
    array_labels = df_feature_vector.columns.to_list()

    df_feature_vector_with_text['index'] = df_feature_vector_with_text.index
    title_array = df_feature_vector_with_text[['id', 'title']]
    title_list = title_array.values.tolist()

    feature_list = df_feature_vector[array_labels].values.tolist()

    ary_1 = feature_list[heimabend_index]

    feature_ary = []
    for index, element in enumerate(feature_list):
        score_cosine = distance.cosine(ary_1, element, weight)
        feature_ary.append([score_cosine])
        # print(score_cosine)

    df_1 = pd.DataFrame(feature_ary, columns=['score'])
    df_2 = pd.DataFrame(title_list, columns=['event_score_id', 'title'])
    df = pd.concat([df_1, df_2], axis=1,).sort_values(by=['score']).iloc[1:15]
    df['event_id'] = df_feature_vector_with_text.iloc[heimabend_index]['id']
    df[['event_id', 'event_score_id', 'score']].to_sql('basic_nextbestheimabend', con=sqlEngine, if_exists='append', index=False)
    print(df[['score', 'title', 'event_score_id', 'event_id']])

pcycling - Reisespiele basteln             220       194
102  0.000044                     Kerzendrehen             173       194
113  0.000046       Wölflingsregeln besprechen             185       194
heimabend_index
117
heimabend_index
        score                              title  event_score_id  event_id
133  0.000032                    Pappteller-Golf             223       198
132  0.000033                    Die Verwandlung             222       198
102  0.000034                       Kerzendrehen             173       198
128  0.000035           Spiel: Der Kaiser mag...             218       198
120  0.000037                       Lichtmalerei             201       198
116  0.000039  Fahrrad-Geschicklichkeits-Parkour             194       198
119  0.000041     Pfadfinder*innen ohne Plastik?             200       198
97   0.000042               Kerzenwachs recyceln             161       198
124  0.000045                        Traumkapsel             205       198
122  0.0000

In [112]:
print('')

       83       180
107  0.000046          Schnitzeljagd mit Abstand             179       180
106  0.000048                    Briefe an Felix             178       180
124  0.000049                        Traumkapsel             205       180
83   0.000050            Rucksack richtig packen             145       180
        score                                title  event_score_id  event_id
114  0.000027  Veganen Brotaufstrich selber machen             186       181
93   0.000048                    Haik-Kekse backen             156       181
110  0.000068                 Hummus selbstgemacht             182       181
104  0.000070                       Digitales Tabu             176       181
97   0.000071                 Kerzenwachs recyceln             161       181
87   0.000072                    Haik-Hackfleisch?             149       181
119  0.000075       Pfadfinder*innen ohne Plastik?             200       181
73   0.000075                    Rotkohl-Farborgel             1

IndexError: list index out of range